In [2]:
from google.cloud import aiplatform

project_id = "udemy-mlops"
region = "us-central1"
staging_bucket="gs://sid-kubeflow-v1"

aiplatform.init(project=project_id, location=region, staging_bucket=staging_bucket)

<h4> Upload the stable version to model registry

In [3]:
display_name = "coupon-recommendation-model"
artifact_uri = "gs://sid-kubeflow-v1/coupon-recommendation/artifacts/"
serving_container_image_uri = "us-docker.pkg.dev/vertex-ai/prediction/xgboost-cpu.1-6:latest"

model = aiplatform.Model.upload(
        display_name=display_name,
        artifact_uri=artifact_uri,
        serving_container_image_uri=serving_container_image_uri,
        sync=False,
        version_aliases=["stable-version"],
        is_default_version=True
    )

model.wait()

Creating Model
Create Model backing LRO: projects/1090925531874/locations/us-central1/models/2527568325047746560/operations/5434893878836592640
Model created. Resource name: projects/1090925531874/locations/us-central1/models/2527568325047746560@1
To use this Model in another session:
model = aiplatform.Model('projects/1090925531874/locations/us-central1/models/2527568325047746560@1')


In [3]:
# DISPLAY_NAME = "coupon-recommendation-model"
# models = aiplatform.Model.list(filter=("display_name={}").format(DISPLAY_NAME))
# parent_model = models[0].resource_name

<h4> Upload the tuned V1 version to the model registry with <b>parent_model</b> parameter

In [4]:
project_id = 1090925531874
model_id = 2527568325047746560
parent_model = 'projects/{}/locations/us-central1/models/{}'.format(project_id,model_id)

new_model = aiplatform.Model.upload(
        parent_model = parent_model,
        artifact_uri="gs://sid-kubeflow-v1/coupon-recommendation/htp-tuned-artifacts/",
        serving_container_image_uri=serving_container_image_uri,
        sync=False,
        version_aliases=["v2"],
        labels={"release":"dev"},
        is_default_version=False
    )
new_model.wait()

Creating Model
Create Model backing LRO: projects/1090925531874/locations/us-central1/models/2527568325047746560/operations/7937206421794324480
Model created. Resource name: projects/1090925531874/locations/us-central1/models/2527568325047746560@2
To use this Model in another session:
model = aiplatform.Model('projects/1090925531874/locations/us-central1/models/2527568325047746560@2')


<h4> Deploy the stable model to an endpoint

In [6]:
deployed_model_display_name = "coupon-model-endpoint"
traffic_split = {"0": 100}
machine_type = "n1-standard-4"
min_replica_count = 1
max_replica_count = 1

endpoint = model.deploy(
        deployed_model_display_name=deployed_model_display_name,
        traffic_split=traffic_split,
        machine_type=machine_type,
        min_replica_count=min_replica_count,
        max_replica_count=max_replica_count
    )

Creating Endpoint
Create Endpoint backing LRO: projects/1090925531874/locations/us-central1/endpoints/4074486627328589824/operations/6930651905077018624
Endpoint created. Resource name: projects/1090925531874/locations/us-central1/endpoints/4074486627328589824
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/1090925531874/locations/us-central1/endpoints/4074486627328589824')
Deploying model to Endpoint : projects/1090925531874/locations/us-central1/endpoints/4074486627328589824
Deploy Endpoint model backing LRO: projects/1090925531874/locations/us-central1/endpoints/4074486627328589824/operations/1166044382042783744
Endpoint model deployed. Resource name: projects/1090925531874/locations/us-central1/endpoints/4074486627328589824


<h4>Run Batch Predictions against stable model version

In [ ]:
gcs_input_uri = 'gs://sid-kubeflow-v1/coupon-recommendation/test-batch.csv'
BUCKET_URI = "gs://sid-kubeflow-v1/coupon-recommendation/batch-prediction-output"

batch_predict_job = model.batch_predict(
    job_display_name="coupon_batch_predict",
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    instances_format="csv",
    predictions_format="jsonl",
    machine_type="n1-standard-4",
    starting_replica_count=1,
    max_replica_count=1,
    sync=False
)

<h4>Run Batch Predictions against V2 model version

In [ ]:
gcs_input_uri = 'gs://sid-kubeflow-v1/coupon-recommendation/test-batch.csv'
BUCKET_URI = "gs://sid-kubeflow-v1/coupon-recommendation/batch-prediction-output"

batch_predict_job = new_model.batch_predict(
    job_display_name="coupon_batch_predict",
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    instances_format="csv",
    predictions_format="jsonl",
    machine_type="n1-standard-4",
    starting_replica_count=1,
    max_replica_count=1,
    sync=False
)